In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import PIL
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from eval.vlc import VLC
from utils.path_utils import Files
from sim.config import TimingConfig, ExperimentConfig
from eval.analysis import Plotter

%matplotlib qt
pd.options.display.max_columns = 30

# Evaluator Testing

In [ ]:
from eval.error_calculator import *
from dataset.bg_extractor import BGExtractor
from eval.vlc import StreamViewer

directory = "D:\\Guy_Gilad\\Exp2_GuyGilad\\raw"
files = Files(directory, extension="Bmp", sorting_key=lambda f: int(f.split("-")[-1].split(".")[0]))
file_list = [f for f in files]

reader = FrameReader(directory, file_list, cv.IMREAD_COLOR)
extractor = BGExtractor(reader)
background = extractor.calc_background(2000, "uniform")

reader = reader.make_stream()

In [ ]:
from utils.io_utils import pickle_load_object, pickle_save_object

pickle_save_object(background, "Exp2_background.np")
# background = pickle_load_object("Exp2_background.np")

In [ ]:
from math import floor, ceil

# use ErrorCalculator.calculate_precise instead

eval = Evaluator(background=background, diff_thresh=20)
progeress = 200


class VL:
    def __init__(self, eval: Evaluator) -> None:
        self.evaluator = eval
        self.progeress = 1

    def update_thresh(self, val: int):
        self.evaluator._diff_thresh = val

    def update_progress(self, val: int):
        self.progeress = val


# vl = VL(eval)
log = pd.read_csv("data/Exp2_GuyGilad_logs_yolo/bboxes2.csv")

viewer = StreamViewer("Viewer")
viewer.open()
# viewer.create_trakbar("threshold", 30, 50, vl.update_thresh)
# viewer.create_trakbar("progress", 0, 100, vl.update_progress)

# viewer.imshow(np.zeros((360, 360)))
i = 0
reader.seek(i)
total_frames = len(reader)

while reader.can_read():
    frame = reader.read()
    wrm_x, wrm_y, wrm_w, wrm_h = log.loc[i, ["wrm_x", "wrm_y", "wrm_w", "wrm_h"]]
    # cam_x, cam_y, cam_w, cam_h = log.loc[i, ["cam_x","cam_y","cam_w","cam_h"]]
    wrm_box = (wrm_x, wrm_y, wrm_w, wrm_h)
    if any([np.isnan(x) for x in wrm_box]):
        reader.progress(progeress)
        i = reader._idx
        continue
    wrm_x = max(0, floor(wrm_x))
    wrm_y = max(0, floor(wrm_y))
    wrm_w = min(ceil(wrm_w), frame.shape[1] - wrm_x)
    wrm_h = min(ceil(wrm_h), frame.shape[0] - wrm_y)

    wrm_box = (wrm_x, wrm_y, wrm_w, wrm_h)
    print(wrm_box)
    wrm_view = frame[wrm_y : wrm_y + wrm_h, wrm_x : wrm_x + wrm_w]
    masked = wrm_view.copy().astype(np.float32)

    mask = eval.find_contour(wrm_view, wrm_box)
    mask = np.where(mask == 0)
    masked[mask] = 0.0
    # masked[mask] = 0
    viewer.imshow(
        masked.astype(np.uint8),
        title=f"frame {i}/{total_frames}  |  thresh={eval._diff_thresh}  |  progress={progeress}",
    )
    # if progress_n_frames > 0:
    reader.progress(progeress)
    i = reader._idx

# Analysis

## VLC

In [ ]:
directory = "D:\\Guy_Gilad\\Exp0_GuyGilad\\raw"
files = Files(directory, extension="Bmp", sorting_key=lambda f: int(f.split("-")[-1].split(".")[0]))
timing_config = TimingConfig.load_json("D:\\Guy_Gilad\\Bio-Proj\\logs\\time_config.json")
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/Optimal/bboxes.csv"
# log_path = "D:\\Guy_Gilad\\Exp2_GuyGilad\\logs_yolo\\bboxes.csv"
# log_path = "data/Exp2_GuyGilad_logs_yolo/bboxes.csv"

In [ ]:
log_path = "data/Exp0_GuyGilad_logs_yolo/bboxes2.csv"
experiment_config = ExperimentConfig.load_json("data/Exp0_GuyGilad_logs_yolo/experiment_config.json")

time_config = TimingConfig(
    imaging_time_ms=100,
    pred_time_ms=15,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.22, 0.22),
    experiment_config=experiment_config,
)
vlc = VLC(files, time_config, log_path, "plt")
vlc.mainloop()

In [ ]:
import cProfile

vlc = VLC(files, timing_config, log_path, "plt")
vlc.print_hotkeys()
vlc.streamer.update_trakbar("delay", 1)
vlc.show_cam = True
vlc.show_micro = True
vlc.show_pred = True
vlc.play = True
vlc.mainloop()
# cProfile.run('vlc.mainloop()')

In [ ]:
vlc = VLC(files, timing_config, log_path, "mic")
vlc.print_hotkeys()
vlc.mainloop()

## Plots

In [ ]:
def cond(df: pd.DataFrame, threshold: float = 1.5) -> pd.DataFrame:
    mask1 = (df["cycle_step"] == 14) & (df["worm_center_dist"] > threshold)
    cycles = df[mask1]["cycle"].unique()
    return df["cycle"].isin(cycles)


def cond_imaging(df: pd.DataFrame) -> pd.DataFrame:
    return df["phase"] == "imaging"


def rolling_avg(data, col, window_size=5):
    data[col + "_avg"] = Plotter.rolling_average(data, window_size=window_size, column=col)
    return data

In [ ]:
path = "data\\Exp2_GuyGilad_logs_yolo\\bboxes.csv"
experiment_config = ExperimentConfig.load_json("data\\Exp2_GuyGilad_logs_yolo\\experiment_config.json")
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.22, 0.22),
    experiment_config=experiment_config,
)
pltr = Plotter(path, time_config)

In [ ]:
log = pd.read_csv(path)

In [ ]:
log.describe()

In [ ]:
log[["cam_w", "cam_h"]] = 368

In [ ]:
pad_x = (log["cam_w"].to_numpy() // 2).reshape(-1, 1)
pad_y = (log["cam_h"].to_numpy() // 2).reshape(-1, 1)
log.loc[:, ["cam_x", "mic_x", "wrm_x"]] = log[["cam_x", "mic_x", "wrm_x"]].values - pad_x
log.loc[:, ["cam_y", "mic_y", "wrm_y"]] = log[["cam_y", "mic_y", "wrm_y"]].values - pad_y

In [ ]:
log.to_csv("bboxes2.csv")

In [ ]:
# original_log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/Optimal/bboxes.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/Controller2/bboxes.csv"
log_path = "/Users/guycohen/Desktop/Bio-Project/PolyfitControllerV2/bboxes.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/test1/bboxes.csv"
timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")

pltr = Plotter(log_path, timing_config)
# pltr_orig = Plotter(original_log_path, timing_config)

In [ ]:
data = pltr.print_statistics(n=10)
# data_orig = pltr_orig.print_statistics()

In [ ]:
(data["mic_center_x"] - data["wrm_center_x"]).hist(bins=15)

In [ ]:
sns.scatterplot(data=data, x="frame", y="worm_deviation", hue="wrm_speed")

In [ ]:
def plot_histogram(pltr, x_col: str, n: int = 1, hue=None, condition=None, **kwargs):
    data = pltr.data_prep_frames(n=n)

    if condition is not None:
        data = data[condition(data)]
    fig, ax = plt.subplots()
    sns.histplot(data=data, x=x_col, hue=hue, stat="density", **kwargs)


def plot_jointplot(
    pltr, x_col: str, y_col: str, n: int = 1, kind: str = "scatter", hue=None, condition=None, transform=None, **kwargs
):
    data = pltr.data_prep_frames(n=n)

    if transform is not None:
        data = transform(data)

    if condition is not None:
        data = data[condition(data)]

    sns.jointplot(data=data, x=x_col, y=y_col, hue=hue, kind=kind, **kwargs)


# plot_histogram(pltr, 'worm_deviation', n=15, hue='worm_deviation')

In [ ]:
plot_jointplot(pltr, "wrm_speed", "worm_deviation", n=5, kind="scatter")

In [ ]:
plot_jointplot(
    pltr,
    x_col="wrm_speed_x",
    y_col="wrm_speed_y",
    n=5,
    kind="scatter",
    hue="worm_angle_avg",
    transform=lambda data: rolling_avg(data, "worm_angle", window_size=5),
)

In [ ]:
plot_histogram(pltr, x_col="bbox_error", condition=lambda df: df["bbox_error"] > 1e-3)

In [ ]:
def avg_speed(data: pd.DataFrame, n_list: list[int] = [10]) -> np.ndarray:
    data = data.copy()

    wrm_speed_x = np.zeros(len(data["frame"]))
    wrm_speed_y = np.zeros(len(data["frame"]))
    for n in n_list:
        wrm_speed_x += data["wrm_center_x"].diff(n) / len(n_list)
        wrm_speed_y += data["wrm_center_y"].diff(n) / len(n_list)
        # avgs.append(Plotter.worm_angle(data, n=n))
    # mean of all averages
    data["wrm_speed_x_avg"] = wrm_speed_x
    data["wrm_speed_y_avg"] = wrm_speed_y
    return data


def worm_angle(data: pd.DataFrame, n: int = 15) -> np.ndarray:
    data = data.copy()

    wrm_speed_x = data["wrm_center_x"].diff(n) / n
    wrm_speed_y = data["wrm_center_y"].diff(n) / n
    angle_speed = np.arctan2(wrm_speed_y, wrm_speed_x)

    min_h, min_w = np.min(data["wrm_h"]), np.min(data["wrm_w"])
    x_sign = np.sign(data["wrm_center_x"].diff(3))
    y_sign = np.sign(data["wrm_center_y"].diff(3))
    # data['wrm_speed_x_avg'] = (data['wrm_w']-min_w)*x_sign
    # data['wrm_speed_y_avg'] = (data['wrm_h']-min_h)*y_sign
    angle = np.arctan2((data["wrm_h"] - min_h) * y_sign, (data["wrm_w"] - min_w) * x_sign)

    angle_diff = np.abs(angle - angle_speed) / (2 * np.pi)
    data["wrm_speed_x_avg"] = wrm_speed_x * (1 - angle_diff)
    data["wrm_speed_y_avg"] = wrm_speed_y * (1 - angle_diff)
    return data

In [ ]:
def plot_flow(
    pltr, x_col: str, y_col: str, n: int = 1, kind: str = "scatter", hue=None, condition=None, transform=None, **kwargs
):
    data = pltr.data_prep_frames(n=n)

    if transform is not None:
        data = transform(data)

    if condition is not None:
        data = data[condition(data)]

    # cmap = sns.color_palette("husl", as_cmap=True, n_colors=len(data['frame']))
    fig, ax = plt.subplots()
    plt.quiver(
        data[x_col],
        data[y_col],
        data["wrm_speed_x_avg"],
        data["wrm_speed_y_avg"],
        np.linspace(0, 1, len(data["frame"])),
        **kwargs
    )
    ax.invert_yaxis()
    ax.grid()


# plot_flow(pltr, x_col='wrm_center_x', y_col='wrm_center_y', n=15, angles='xy', transform=lambda data: rolling_avg(data, 'worm_angle', window_size=1))
plot_flow(
    pltr,
    x_col="wrm_center_x",
    y_col="wrm_center_y",
    n=5,
    angles="xy",
    scale_units="xy",
    scale=1,
    transform=lambda df: worm_angle(df, 15),
    #   condition=lambda df: df['cycle_step'] == 9,
)

In [ ]:
# cols = ['frame', 'cycle', 'plt_x', 'plt_y', 'cam_x', 'cam_y', 'cam_w',
#        'cam_h', 'mic_x', 'mic_y', 'mic_w', 'mic_h', 'wrm_x', 'wrm_y', 'wrm_w',
#        'wrm_h', 'wrm_center_x', 'wrm_center_y', 'mic_center_x', 'mic_center_y',
#        'wrm_speed', 'bbox_error', 'bbox_edge_diff']


# (data[cols] - data_orig[cols]).describe()

In [ ]:
pltr.plot_area_vs_speed()

In [ ]:
data.columns

In [ ]:
pltr.plot_2d_deviation(n=2, hue="cycle_step")

In [ ]:
pltr.plot_deviation()

In [ ]:
pltr.plot_area_vs_speed_guy(n=1, window_size=1, hue="phase")

In [ ]:
pltr.plot_cycle_step_vs_speed()

In [ ]:
pltr.plot_trajectory(n=15, hue="wrm_speed")

In [ ]:
def plot_speed_vs_time(pltr, n: int = 1, window_size: int = 15, hue=None, condition=None) -> plt.Figure:
    data = pltr.data_prep_frames(n=n)
    data["wrm_speed_avg"] = Plotter.rolling_average(data, window_size=window_size, column="wrm_speed")
    fig, ax = plt.subplots()
    if condition is not None:
        mask = condition(data) & mask
    g = sns.histplot(data=data, x="wrm_speed", ax=ax, hue=hue)
    g.figure.suptitle(f"n = {n}, rolling window = {window_size}")
    return g.figure

In [ ]:
plot_speed_vs_time(pltr, n=15, window_size=1, hue="phase")

In [ ]:
timing_config.mm_per_px / timing_config.ms_per_frame

In [ ]:
def plot_2d_deviation_with_angle(pltr, n: int = 1, hue="cycle_step", condition=None) -> plt.Figure:
    data = pltr.data_prep_frames(n=n)
    data["worm_center_dist_x"] = data["wrm_center_x"] - data["mic_center_x"]
    data["worm_center_dist_y"] = data["wrm_center_y"] - data["mic_center_y"]

    data["cycle_step"] = data["frame"] % pltr.timing_config.cycle_frame_num
    min_h, min_w = np.min(data["wrm_h"]), np.min(data["wrm_w"])
    x_sign = np.sign(data["wrm_center_x"].diff(n))
    y_sign = np.sign(data["wrm_center_y"].diff(n))
    data["angle"] = np.arctan2((data["wrm_h"] - min_h) * y_sign, (data["wrm_w"] - min_w) * x_sign)
    if condition is not None:
        data = data[condition(data)]

    # display(data.head(50))
    # g = sns.histplot(data=data, x="angle")
    g = sns.jointplot(data=data, x="wrm_speed", y="angle", kind="scatter", hue=hue, alpha=0.6)
    # g = sns.jointplot(data=data, x="worm_center_dist_x", y="worm_center_dist_y", kind="scatter", hue=hue, alpha=0.6)
    # g.set_axis_labels("distance x", "distance y")
    # g.figure.suptitle(f"distance between worm and microscope centers in each axis")

    return g.figure

In [ ]:
plot_2d_deviation_with_angle(pltr, 5, hue=None, condition=lambda df: df["wrm_speed"] > -0.3)

In [ ]:
def plot_area_vs_time(pltr: Plotter, n: int = 1, window_size: int = 15, hue=None, condition=None) -> plt.Figure:
    data = pltr.data_prep_frames(n=n)
    data["wrm_speed_avg"] = Plotter.rolling_average(data, window_size=window_size, column="wrm_speed")
    data["worm_center_dist"] = np.sqrt(
        (data["wrm_center_x"] - data["mic_center_x"]) ** 2 + (data["wrm_center_y"] - data["mic_center_y"]) ** 2
    )
    data["cycle_step"] = data["frame"] % pltr.timing_config.cycle_frame_num
    data["angle"] = np.arctan2(data["wrm_w"], data["wrm_h"])
    # fig, ax = plt.subplots()
    mask = data["bbox_error"] > 1e-3
    if condition is not None:
        mask = condition(data) & mask
    g = sns.jointplot(data=data[mask], x="wrm_speed_avg", y="worm_center_dist", hue=hue, kind="scatter", dropna=True)
    g.figure.suptitle(f"n = {n}, rolling window = {window_size}")
    return g.figure

In [ ]:
plot_area_vs_time(pltr, n=18, window_size=1, hue="angle")

In [ ]:
raise Exception("Finished")

# Run Simulation

In [ ]:
from sim import *
from sim.sim_controllers import *
from sim.config import ExperimentConfig

# base_path = "/Users/guycohen/Desktop/Bio-Project/"
base_path = f"data/Exp1_GuyGilad_logs_yolo/"

## Run CSV Simulation

In [ ]:
exp_base_path = "logs"

# time_config = TimingConfig.load_json(base_path + "time_config.json")
# time_config = TimingConfig.load_json(base_path + "time_config_medium_micro.json")
experiment_config = ExperimentConfig.load_json(f"{base_path}/experiment_config.json")
log_config = LogConfig(
    root_folder=exp_base_path,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

In [ ]:
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)

In [ ]:
from sim.sim_controllers import *
import torch
from neural.config import TrainConfig

log_path = base_path + "bboxes2.csv"

# controller = CsvController(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")

# controller = TestController1(time_config, "/Users/guycohen/Desktop/Bio-Project/eval/bboxes.csv")
# controller = Controller2(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")
# controller = OptimalController(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")

train_config = TrainConfig.load_pickle("runs/May23_13-10-02_MacbookGuy.local/train_config.pkl")

model = torch.load("runs/May23_13-10-02_MacbookGuy.local/RMLP4")
print(model)
controller = MLPController(time_config, log_path, model, train_config.dataset)

weights = np.ones(11)
# weights = np.asanyarray([0.0477, 0.0002, 0. ,    0.0007, 0.0057, 0.042 , 0.1558, 0.6245 ,0.7627])
sample_frames = np.asanyarray(
    [
        0,
        5,
        9,
        -time_config.cycle_frame_num + 0,
        -time_config.cycle_frame_num + 5,
        -time_config.cycle_frame_num + 9,
        -2 * time_config.cycle_frame_num + 0,
        -2 * time_config.cycle_frame_num + 5,
        -2 * time_config.cycle_frame_num + 9,
        -3 * time_config.cycle_frame_num + 9,
        -4 * time_config.cycle_frame_num + 9,
    ]
)
# controller = PolyfitController(time_config, log_path, degree=2, weights=weights, sample_times=sample_frames)

In [ ]:
logging_controller = LoggingController(controller, log_config)
motor_controller = SineMotorController(time_config)
# motor_controller = SimpleMovementController(time_config, 0)

sim = Simulator(time_config, experiment_config, logging_controller, reader=None, motor_controller=motor_controller)

In [ ]:
sim.run()
time_config.save_json(exp_base_path + "/time_config.json")
experiment_config.save_json(exp_base_path + "/exp_config.json")

In [ ]:
from eval.analysis import Plotter

pltr = Plotter(log_config.bbox_file_path, time_config)
data = pltr.print_statistics(n=10)

In [ ]:
def bbox_eval(pltr: Plotter, df: pd.DataFrame) -> pd.DataFrame:
    data = pltr.data_prep_frames(n=10)
    grouped_data = data.groupby("cycle")
    avg_speed = grouped_data["wrm_speed"].mean()

    data = Plotter.remove_phase(data, "moving")

    max_area_diff = data.groupby("cycle")["bbox_error"].max()

    data = pd.DataFrame({"wrm_speed": avg_speed, "bbox_error": max_area_diff})
    return data

In [ ]:
pltr.plot_histogram(
    "bbox_error",
    n=10,
    stat="density",
    transform=lambda df: bbox_eval(pltr, df),
    condition=lambda df: df["bbox_error"] > 1e-2,
)

In [ ]:
pltr.plot_area_vs_speed()

In [ ]:
_ = pltr.plot_deviation()

In [ ]:
raise Exception("Finished")

## Real Simulation

In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [ ]:
controller = LoggingController(time_config, yolo_config, log_config)
# controller = YoloController(time_config, yolo_config)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run()

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")